# Unit Testing



In [ ]:
import sys
from pathlib import Path

current = Path.cwd()
for parent in [current, *current.parents]:
    if (parent / '_config.yml').exists():
        project_root = parent  # ← Add project root, not chapters
        break
else:
    project_root = Path.cwd().parent.parent

sys.path.insert(0, str(project_root))

from shared import thinkpython, diagram, jupyturtle
from shared.download import download

# Register as top-level modules so direct imports work in subsequent cells
sys.modules['thinkpython'] = thinkpython
sys.modules['diagram'] = diagram
sys.modules['jupyturtle'] = jupyturtle

## Unit Testing: Proactive Debugging

Unit testing is a powerful debugging technique that prevents errors before they occur. Instead of waiting for bugs to appear and then hunting them down, you write tests that verify your code works correctly from the start.

**Key Benefits of Unit Testing:**
- **Catch bugs early**: Find problems before users do
- **Regression prevention**: Ensure fixes don't break existing functionality  
- **Documentation**: Tests show how your code should behave
- **Confidence**: Make changes knowing tests will catch problems
- **Systematic verification**: Methodically check all code paths

**Basic Testing Concepts:**
- **Test case**: A single test that checks one specific behavior
- **Test suite**: A collection of related test cases
- **Assertion**: A statement that checks if a condition is true
- **Test runner**: Tool that executes tests and reports results

Python's built-in `unittest` module provides everything you need to write and run tests.

In [ ]:
import unittest

# Example: Testing a simple function with unit tests

def calculate_grade(score, total_points):
    """Calculate percentage grade from score and total points."""
    if total_points <= 0:
        raise ValueError("Total points must be positive")
    if score < 0:
        raise ValueError("Score cannot be negative") 
    if score > total_points:
        raise ValueError("Score cannot exceed total points")
    
    percentage = (score / total_points) * 100
    return round(percentage, 2)

def get_letter_grade(percentage):
    """Convert percentage to letter grade."""
    if percentage >= 90:
        return 'A'
    elif percentage >= 80:
        return 'B'
    elif percentage >= 70:
        return 'C'
    elif percentage >= 60:
        return 'D'
    else:
        return 'F'

# Unit tests for our functions
class TestGradeCalculation(unittest.TestCase):
    """Test cases for grade calculation functions."""
    
    def test_calculate_grade_normal_cases(self):
        """Test normal grade calculations."""
        self.assertEqual(calculate_grade(85, 100), 85.0)
        self.assertEqual(calculate_grade(95, 100), 95.0)
        self.assertEqual(calculate_grade(50, 100), 50.0)
        self.assertEqual(calculate_grade(0, 100), 0.0)
    
    def test_calculate_grade_edge_cases(self):
        """Test edge cases for grade calculation."""
        self.assertEqual(calculate_grade(100, 100), 100.0)
        self.assertEqual(calculate_grade(87, 92), 94.57)
    
    def test_calculate_grade_invalid_input(self):
        """Test that invalid inputs raise appropriate exceptions."""
        with self.assertRaises(ValueError):
            calculate_grade(85, 0)  # Zero total points
        
        with self.assertRaises(ValueError):
            calculate_grade(-10, 100)  # Negative score
        
        with self.assertRaises(ValueError):
            calculate_grade(110, 100)  # Score exceeds total
    
    def test_letter_grade_conversion(self):
        """Test letter grade assignments."""
        self.assertEqual(get_letter_grade(95), 'A')
        self.assertEqual(get_letter_grade(85), 'B')
        self.assertEqual(get_letter_grade(75), 'C')
        self.assertEqual(get_letter_grade(65), 'D')
        self.assertEqual(get_letter_grade(55), 'F')
        
        # Test boundary conditions
        self.assertEqual(get_letter_grade(90), 'A')
        self.assertEqual(get_letter_grade(89.9), 'B')

# Run the tests
if __name__ == '__main__':
    # Run tests and capture results
    import io
    import sys
    
    # Capture test output
    test_output = io.StringIO()
    runner = unittest.TextTestRunner(stream=test_output, verbosity=2)
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGradeCalculation)
    result = runner.run(suite)
    
    # Display results
    print("=== Unit Test Results ===")
    print(test_output.getvalue())
    
    if result.wasSuccessful():
        print("✅ All tests passed!")
    else:
        print(f"❌ {len(result.failures)} test(s) failed")
        print(f"❌ {len(result.errors)} error(s) occurred")

In [ ]:
# Example: Test-Driven Debugging Workflow

def find_maximum(numbers):
    """Find the maximum value in a list of numbers."""
    # Initial implementation with a bug
    if not numbers:
        return None
    
    max_val = numbers[0]
    for num in numbers:
        if num > max_val:
            max_val = num
    return max_val

# Write tests first to define expected behavior
class TestFindMaximum(unittest.TestCase):
    """Tests for find_maximum function."""
    
    def test_normal_cases(self):
        """Test with normal lists of numbers."""
        self.assertEqual(find_maximum([1, 5, 3, 9, 2]), 9)
        self.assertEqual(find_maximum([10, 20, 15]), 20)
        self.assertEqual(find_maximum([7]), 7)
    
    def test_negative_numbers(self):
        """Test with negative numbers."""
        self.assertEqual(find_maximum([-1, -5, -3]), -1)
        self.assertEqual(find_maximum([-10, 5, -3]), 5)
    
    def test_duplicates(self):
        """Test with duplicate maximum values."""
        self.assertEqual(find_maximum([5, 5, 5]), 5)
        self.assertEqual(find_maximum([1, 3, 3, 2]), 3)
    
    def test_empty_list(self):
        """Test with empty list."""
        self.assertIsNone(find_maximum([]))
    
    def test_edge_cases(self):
        """Test edge cases."""
        self.assertEqual(find_maximum([0]), 0)
        self.assertEqual(find_maximum([0, 0, 0]), 0)

# Demonstrate the test-driven debugging process
print("=== Test-Driven Debugging Example ===")
print("1. First, let's run our tests to see if our function works:")

# Run tests to find bugs
suite = unittest.TestLoader().loadTestsFromTestCase(TestFindMaximum)
test_output = io.StringIO()
runner = unittest.TextTestRunner(stream=test_output, verbosity=2)
result = runner.run(suite)

print(test_output.getvalue())

if result.wasSuccessful():
    print("✅ All tests pass! Our function is working correctly.")
else:
    print("❌ Some tests failed. Let's debug using the test failures.")
    
    # Show what manual testing would look like
    print("\n2. Manual debugging based on test failures:")
    test_cases = [
        ([1, 5, 3, 9, 2], 9),
        ([-1, -5, -3], -1),
        ([]),
    ]
    
    for numbers, expected in test_cases:
        result = find_maximum(numbers)
        status = "✅" if result == expected else "❌"
        print(f"{status} find_maximum({numbers}) = {result}, expected: {expected}")

print("\n3. The tests help us verify our function works correctly!")
print("   If there were bugs, the failing tests would show us exactly what to fix.")

In [ ]:
# Example: Using Tests to Isolate and Fix Bugs

class Calculator:
    """A simple calculator class with some bugs to find."""
    
    def add(self, a, b):
        return a + b
    
    def subtract(self, a, b):
        return a - b
    
    def multiply(self, a, b):
        return a * b
    
    def divide(self, a, b):
        # Bug: Not handling division by zero
        return a / b
    
    def power(self, base, exponent):
        # Bug: Not handling negative bases correctly
        if base < 0:
            return "Error: negative base"
        return base ** exponent

# Comprehensive tests to find the bugs
class TestCalculator(unittest.TestCase):
    """Tests to find bugs in Calculator class."""
    
    def setUp(self):
        """Set up test calculator instance."""
        self.calc = Calculator()
    
    def test_addition(self):
        """Test addition operation."""
        self.assertEqual(self.calc.add(2, 3), 5)
        self.assertEqual(self.calc.add(-1, 1), 0)
        self.assertEqual(self.calc.add(0, 0), 0)
    
    def test_subtraction(self):
        """Test subtraction operation."""
        self.assertEqual(self.calc.subtract(5, 3), 2)
        self.assertEqual(self.calc.subtract(1, 1), 0)
        self.assertEqual(self.calc.subtract(-1, -1), 0)
    
    def test_multiplication(self):
        """Test multiplication operation."""
        self.assertEqual(self.calc.multiply(3, 4), 12)
        self.assertEqual(self.calc.multiply(-2, 3), -6)
        self.assertEqual(self.calc.multiply(0, 100), 0)
    
    def test_division_normal(self):
        """Test normal division cases."""
        self.assertEqual(self.calc.divide(10, 2), 5)
        self.assertEqual(self.calc.divide(9, 3), 3)
        self.assertAlmostEqual(self.calc.divide(1, 3), 0.333333, places=5)
    
    def test_division_by_zero(self):
        """Test division by zero - should raise exception."""
        with self.assertRaises(ZeroDivisionError):
            self.calc.divide(10, 0)
    
    def test_power_positive(self):
        """Test power with positive bases."""
        self.assertEqual(self.calc.power(2, 3), 8)
        self.assertEqual(self.calc.power(5, 0), 1)
        self.assertEqual(self.calc.power(1, 100), 1)
    
    def test_power_negative_base(self):
        """Test power with negative bases."""
        # This test will fail due to our bug
        self.assertEqual(self.calc.power(-2, 2), 4)
        self.assertEqual(self.calc.power(-3, 3), -27)

# Run tests to identify bugs
print("=== Bug Detection with Unit Tests ===")
print("Running tests to identify bugs in Calculator class...")

suite = unittest.TestLoader().loadTestsFromTestCase(TestCalculator)
test_output = io.StringIO()
runner = unittest.TextTestRunner(stream=test_output, verbosity=2)
result = runner.run(suite)

print(test_output.getvalue())

print(f"\n📊 Test Summary:")
print(f"   Tests run: {result.testsRun}")
print(f"   Failures: {len(result.failures)}")
print(f"   Errors: {len(result.errors)}")

if result.failures:
    print(f"\n🐛 Bugs found through testing:")
    for test, traceback in result.failures:
        print(f"   • {test}: Test revealed a bug in the implementation")

if result.errors:
    print(f"\n💥 Errors found through testing:")
    for test, traceback in result.errors:
        print(f"   • {test}: Test revealed an error condition")

print(f"\n✨ This demonstrates how unit tests systematically find bugs!")
print(f"   Without tests, these bugs might go unnoticed until production.")

### Unit Testing Best Practices for Debugging

**1. Write Tests First (Test-Driven Development)**
- Define expected behavior before implementing
- Tests serve as specifications
- Easier to catch bugs early

**2. Test Edge Cases and Error Conditions**
- Empty inputs, boundary values, invalid data
- Exception handling scenarios
- Unusual but valid inputs

**3. Use Descriptive Test Names**
- `test_divide_by_zero_raises_exception` vs `test_division`
- Makes it clear what behavior is being tested
- Easier to understand failures

**4. Organize Tests Logically**
- Group related tests in test classes
- Use `setUp()` and `tearDown()` methods for common setup
- Keep tests independent of each other

**5. Assert Specific Behaviors**
- Use appropriate assertion methods (`assertEqual`, `assertRaises`, etc.)
- Test one behavior per test method
- Make assertions clear and specific

**Unit Testing as Debugging Strategy:**
- **Prevention**: Catch bugs before they happen
- **Isolation**: Quickly identify which component has the problem
- **Regression**: Ensure fixes don't break existing functionality
- **Documentation**: Tests show expected behavior
- **Confidence**: Make changes knowing tests will catch issues

# Testing

In [ ]:
def uses_any(word, letters):
    """Checks if a word uses any of a list of letters.
    
    >>> uses_any('banana', 'aeiou')
    True
    >>> uses_any('apple', 'xyz')
    False
    """
    for letter in word.lower():
        if letter in letters.lower():
            return True
    return False

Each test begins with `>>>`, which is used as a prompt in some Python environments to indicate where the user can type code.
In a doctest, the prompt is followed by an expression, usually a function call.
The following line indicates the value the expression should have if the function works correctly.

In the first example, `'banana'` uses `'a'`, so the result should be `True`.
In the second example, `'apple'` does not use any of `'xyz'`, so the result should be `False`.

To run these tests, we have to import the `doctest` module and run a function called `run_docstring_examples`.
To make this function easier to use, I wrote the following function, which takes a function object as an argument.

In [ ]:
from doctest import run_docstring_examples

def run_doctests(func):
    run_docstring_examples(func, globals(), name=func.__name__)

We haven't learned about `globals` and `__name__` yet -- you can ignore them.
Now we can test `uses_any` like this.

In [ ]:
run_doctests(uses_any)

`run_doctests` finds the expressions in the docstring and evaluates them.
If the result is the expected value, the test **passes**.
Otherwise it **fails**.

If all tests pass, `run_doctests` displays no output -- in that case, no news is good news.
To see what happens when a test fails, here's an incorrect version of `uses_any`.

In [ ]:
def uses_any_incorrect(word, letters):
    """Checks if a word uses any of a list of letters.
    
    >>> uses_any_incorrect('banana', 'aeiou')
    True
    >>> uses_any_incorrect('apple', 'xyz')
    False
    """
    for letter in word.lower():
        if letter in letters.lower():
            return True
        else:
            return False     # INCORRECT!

And here's what happens when we test it.

In [ ]:
run_doctests(uses_any_incorrect)

**********************************************************************
File "__main__", line 4, in uses_any_incorrect
Failed example:
    uses_any_incorrect('banana', 'aeiou')
Expected:
    True
Got:
    False


The output includes the example that failed, the value the function was expected to produce, and the value the function actually produced.

If you are not sure why this test failed, you'll have a chance to debug it as an exercise.